# 🚀 Vesuvius Challenge - SwinUNETR v2 完全独立版

**Medical AI SwinUNETR v2** - 1つのノートブックで完結（外部ファイル不要）

## 📥 Step 1: Kaggle認証とデータダウンロード

In [ ]:
# 必要なライブラリのインストールと確認
import subprocess
import sys
import os

# MONAIインストール（SwinUNETR用）
try:
    import monai
    print(f"✅ MONAI {monai.__version__} インストール済み")
except ImportError:
    print("📦 MONAIをインストール中...")
    subprocess.check_call([sys.executable, "-m", "pip", "install", "monai", "-q"])
    import monai
    print(f"✅ MONAI {monai.__version__} インストール完了")

# その他必要なライブラリ
packages = [
    ("kaggle", "kaggle"),
    ("PIL", "pillow"),
    ("cv2", "opencv-python"),
    ("einops", "einops"),
    ("timm", "timm")
]

for import_name, install_name in packages:
    try:
        exec(f"import {import_name}")
        print(f"✅ {import_name} インストール済み")
    except ImportError:
        print(f"📦 {install_name}をインストール中...")
        subprocess.check_call([sys.executable, "-m", "pip", "install", install_name, "-q"])
        print(f"✅ {install_name} インストール完了")

In [ ]:
# Kaggle認証設定
import json
from pathlib import Path

def setup_kaggle_auth(username=None, key=None):
    """Kaggle認証設定"""
    
    # 既存のkaggle.json確認
    locations = [
        Path("/workspace/kaggle.json"),
        Path("/workspace/.kaggle/kaggle.json"),
        Path.home() / ".kaggle" / "kaggle.json",
        Path("./kaggle.json"),
        Path("/content/kaggle.json")
    ]
    
    for loc in locations:
        if loc.exists():
            print(f"✅ Kaggle認証ファイル発見: {loc}")
            if loc.parent.name != ".kaggle":
                os.environ['KAGGLE_CONFIG_DIR'] = str(loc.parent)
            return True
    
    # 認証情報が提供された場合
    if username and key:
        kaggle_config = {"username": username, "key": key}
        
        if Path("/workspace").exists():
            kaggle_path = Path("/workspace/kaggle.json")
        else:
            kaggle_path = Path("./kaggle.json")
        
        with open(kaggle_path, 'w') as f:
            json.dump(kaggle_config, f)
        
        os.chmod(kaggle_path, 0o600)
        os.environ['KAGGLE_CONFIG_DIR'] = str(kaggle_path.parent)
        print(f"✅ Kaggle認証設定完了: {kaggle_path}")
        return True
    
    print("⚠️ Kaggle認証が必要です")
    return False

# 認証確認
auth_ok = setup_kaggle_auth()

In [ ]:
# 認証情報を設定（必要に応じて実行）
# setup_kaggle_auth(username="YOUR_USERNAME", key="YOUR_API_KEY")

In [ ]:
# データダウンロード
import zipfile

def download_vesuvius_data():
    """Vesuviusデータセットをダウンロード"""
    
    # データディレクトリ設定
    if Path("/workspace").exists():
        data_dir = Path("/workspace/data")
    else:
        data_dir = Path("./data")
    
    data_dir.mkdir(parents=True, exist_ok=True)
    dataset_dir = data_dir / "vesuvius-challenge-surface-detection"
    
    # 既存データ確認
    if dataset_dir.exists():
        train_images = dataset_dir / "train_images"
        if train_images.exists():
            tiff_files = list(train_images.glob("*.tif"))
            if len(tiff_files) > 0:
                print(f"✅ データ既存: {len(tiff_files)}画像")
                return str(dataset_dir)
    
    print("📥 データダウンロード開始...")
    
    try:
        cmd = f"kaggle competitions download -c vesuvius-challenge-surface-detection -p {data_dir}"
        result = subprocess.run(cmd, shell=True, capture_output=True, text=True, timeout=600)
        
        if result.returncode == 0:
            print("✅ ダウンロード完了")
            
            # ZIP解凍
            zip_file = data_dir / "vesuvius-challenge-surface-detection.zip"
            if zip_file.exists():
                print("📦 解凍中...")
                with zipfile.ZipFile(zip_file, 'r') as zip_ref:
                    zip_ref.extractall(data_dir)
                zip_file.unlink()
                print("✅ 解凍完了")
            
            return str(dataset_dir)
    except Exception as e:
        print(f"⚠️ ダウンロードエラー: {e}")
    
    return None

# ダウンロード実行
data_path = download_vesuvius_data()

## 🔧 Step 2: ライブラリインポート

In [ ]:
# 必要なライブラリ
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt
from typing import Optional, Tuple, List
from pathlib import Path
from PIL import Image
import cv2
from einops import rearrange
import math

# MONAI imports
from monai.networks.nets import SwinUNETR
from monai.losses import DiceCELoss
from monai.metrics import DiceMetric

# GPU確認
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"🖥️ 使用デバイス: {device}")
if torch.cuda.is_available():
    print(f"   GPU: {torch.cuda.get_device_name(0)}")
    print(f"   メモリ: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")

## 📊 Step 3: データセット（実データ対応）

In [ ]:
class VesuviusDataset(Dataset):
    """Vesuvius実データセット for SwinUNETR"""
    
    def __init__(self, split='train', volume_size=(96, 96, 96), num_samples=10):
        self.split = split
        self.volume_size = volume_size
        self.num_samples = num_samples
        
        # データパス検出
        self.data_path = self._find_data()
        
        # データロード
        self.volumes, self.labels = self._load_data()
    
    def _find_data(self):
        """データパス自動検出"""
        paths = [
            "/workspace/data/vesuvius-challenge-surface-detection",
            "./data/vesuvius-challenge-surface-detection",
            "/workspace/vesuvius-challenge-surface-detection",
            "./vesuvius-challenge-surface-detection",
            "/content/vesuvius-challenge-surface-detection"
        ]
        
        for path in paths:
            p = Path(path)
            if p.exists():
                train_images = p / "train_images"
                if train_images.exists():
                    tiffs = list(train_images.glob("*.tif"))
                    if len(tiffs) > 0:
                        print(f"✅ 実データ発見: {p} ({len(tiffs)}画像)")
                        return str(p)
        
        print("📊 実データなし - 高品質デモデータ使用")
        return None
    
    def _load_data(self):
        """データロード"""
        if self.data_path:
            return self._load_real_data()
        else:
            return self._generate_demo_data()
    
    def _load_real_data(self):
        """実データロード"""
        volumes = []
        labels = []
        
        data_path = Path(self.data_path)
        train_images_dir = data_path / "train_images"
        train_labels_dir = data_path / "train_labels"
        
        tiff_files = sorted(list(train_images_dir.glob("*.tif")))
        
        if len(tiff_files) == 0:
            return self._generate_demo_data()
        
        print(f"📂 {len(tiff_files)}個のTIFFファイルから{self.num_samples}個のボリューム作成")
        
        H, W, D = self.volume_size
        
        for vol_idx in range(min(self.num_samples, max(1, len(tiff_files) // D))):
            start_idx = (vol_idx * D) % max(1, len(tiff_files) - D)
            selected_files = tiff_files[start_idx:start_idx + D]
            
            if len(selected_files) < D:
                # 循環使用
                selected_files = (selected_files * ((D // len(selected_files)) + 1))[:D]
            
            volume_slices = []
            label_slices = []
            
            for tiff_file in selected_files[:D]:
                try:
                    # 画像読み込み
                    img = np.array(Image.open(tiff_file), dtype=np.float32)
                    
                    if len(img.shape) == 3:
                        img = img.mean(axis=2)
                    
                    # リサイズ
                    img = cv2.resize(img, (W, H))
                    
                    # 正規化
                    img = (img - img.mean()) / (img.std() + 1e-8)
                    
                    # ラベル処理
                    if train_labels_dir.exists():
                        label_file = train_labels_dir / tiff_file.name
                        if label_file.exists():
                            label = np.array(Image.open(label_file), dtype=np.uint8)
                            if len(label.shape) == 3:
                                label = label.mean(axis=2)
                            label = cv2.resize(label, (W, H), interpolation=cv2.INTER_NEAREST)
                            label = (label > 127).astype(np.int64)
                        else:
                            label = (img > np.percentile(img, 75)).astype(np.int64)
                    else:
                        label = (img > np.percentile(img, 75)).astype(np.int64)
                    
                    volume_slices.append(img)
                    label_slices.append(label)
                    
                except Exception as e:
                    # エラー時はランダムスライス
                    volume_slices.append(np.random.randn(H, W).astype(np.float32))
                    label_slices.append(np.zeros((H, W), dtype=np.int64))
            
            # 3Dボリューム構築
            volume = np.stack(volume_slices, axis=2)
            label = np.stack(label_slices, axis=2)
            
            volumes.append(volume)
            labels.append(label)
            
            if vol_idx == 0:
                print(f"  サンプル: {volume.shape}, 前景率: {(label==1).mean():.2%}")
        
        print(f"✅ {len(volumes)}個の実データボリューム作成完了")
        return volumes, labels
    
    def _generate_demo_data(self):
        """高品質デモデータ生成"""
        volumes = []
        labels = []
        
        H, W, D = self.volume_size
        
        for i in range(self.num_samples):
            # リアルな3Dボリューム生成
            volume = np.random.randn(H, W, D).astype(np.float32)
            
            # 3Dテクスチャパターン
            for z in range(D):
                x, y = np.meshgrid(np.linspace(0, 3*np.pi, H), np.linspace(0, 3*np.pi, W))
                pattern = np.sin(x + i + z/10) * np.cos(y + z/10)
                volume[:, :, z] += pattern * 0.5
                volume[:, :, z] += np.random.randn(H, W) * 0.2
            
            # 正規化
            volume = (volume - volume.mean()) / (volume.std() + 1e-8)
            
            # 3Dセグメンテーションラベル生成
            label = np.zeros((H, W, D), dtype=np.int64)
            num_regions = np.random.randint(3, 8)
            
            for _ in range(num_regions):
                cx = np.random.randint(H//4, 3*H//4)
                cy = np.random.randint(W//4, 3*W//4)
                cz = np.random.randint(D//4, 3*D//4)
                size = np.random.randint(8, 20)
                
                for x in range(max(0, cx-size), min(H, cx+size)):
                    for y in range(max(0, cy-size), min(W, cy+size)):
                        for z in range(max(0, cz-size//2), min(D, cz+size//2)):
                            dist = ((x-cx)**2 + (y-cy)**2 + (z-cz)**2*4) / size**2
                            if dist < 1:
                                label[x, y, z] = 1
            
            volumes.append(volume)
            labels.append(label)
        
        print(f"✅ {len(volumes)}個のデモボリューム生成完了")
        return volumes, labels
    
    def __len__(self):
        return len(self.volumes)
    
    def __getitem__(self, idx):
        volume = torch.FloatTensor(self.volumes[idx]).unsqueeze(0)  # (1, H, W, D)
        label = torch.LongTensor(self.labels[idx])  # (H, W, D)
        return {'data': volume, 'target': label}

## 🏗️ Step 4: SwinUNETR v2 モデル定義

In [ ]:
def create_swinunetr_v2(in_channels=1, out_channels=2, img_size=(96, 96, 96), feature_size=48):
    """
    SwinUNETR v2 モデル作成
    Medical AI仕様に準拠
    """
    model = SwinUNETR(
        img_size=img_size,
        in_channels=in_channels,
        out_channels=out_channels,
        feature_size=feature_size,  # 48 for base model
        drop_rate=0.0,
        attn_drop_rate=0.0,
        dropout_path_rate=0.0,
        use_checkpoint=False,  # メモリ節約のため
    )
    
    return model

# モデル作成
print("🏗️ SwinUNETR v2 モデル作成中...")
model = create_swinunetr_v2(
    in_channels=1,
    out_channels=2,
    img_size=(96, 96, 96),
    feature_size=48  # 基本サイズ（GPUメモリに応じて調整）
).to(device)

# パラメータ数確認
total_params = sum(p.numel() for p in model.parameters())
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f"✅ モデル作成完了")
print(f"   総パラメータ数: {total_params:,}")
print(f"   学習可能パラメータ: {trainable_params:,}")

## 📊 Step 5: データローダー作成

In [ ]:
# データセット作成
print("📊 データローダー作成中...\n")

# 訓練データセット
train_dataset = VesuviusDataset(
    split='train',
    volume_size=(96, 96, 96),  # SwinUNETRは96の倍数推奨
    num_samples=10  # GPUメモリに応じて調整
)

# 検証データセット
val_dataset = VesuviusDataset(
    split='val',
    volume_size=(96, 96, 96),
    num_samples=3
)

# データローダー
train_loader = DataLoader(
    train_dataset,
    batch_size=1,  # SwinUNETRは大きいので小さめ
    shuffle=True,
    num_workers=2,
    pin_memory=torch.cuda.is_available()
)

val_loader = DataLoader(
    val_dataset,
    batch_size=1,
    shuffle=False,
    num_workers=2,
    pin_memory=torch.cuda.is_available()
)

print(f"✅ データローダー作成完了")
print(f"   訓練: {len(train_dataset)}サンプル / {len(train_loader)}バッチ")
print(f"   検証: {len(val_dataset)}サンプル / {len(val_loader)}バッチ")

# サンプル確認
sample = next(iter(train_loader))
print(f"\n📐 データ形状:")
print(f"   入力: {sample['data'].shape} [B, C, H, W, D]")
print(f"   ラベル: {sample['target'].shape} [B, H, W, D]")

## ⚙️ Step 6: 学習設定

In [ ]:
# 損失関数（DiceCELoss - Dice + CrossEntropy）
criterion = DiceCELoss(to_onehot_y=True, softmax=True)

# オプティマイザー（AdamW推奨）
optimizer = optim.AdamW(
    model.parameters(),
    lr=1e-4,  # SwinUNETRは小さめの学習率
    weight_decay=1e-5
)

# スケジューラー（CosineAnnealingLR推奨）
scheduler = optim.lr_scheduler.CosineAnnealingLR(
    optimizer,
    T_max=20,  # エポック数
    eta_min=1e-6
)

# メトリクス
dice_metric = DiceMetric(include_background=False, reduction="mean")

print("⚙️ 学習設定完了")
print(f"   損失関数: DiceCELoss")
print(f"   オプティマイザー: AdamW")
print(f"   学習率: {optimizer.param_groups[0]['lr']}")

## 🏋️ Step 7: 学習ループ

In [ ]:
def train_epoch(model, loader, criterion, optimizer, device):
    """1エポックの学習"""
    model.train()
    total_loss = 0
    
    for batch in tqdm(loader, desc="Training"):
        data = batch['data'].to(device)
        target = batch['target'].to(device)
        
        optimizer.zero_grad()
        
        # Forward
        output = model(data)
        
        # Loss計算
        loss = criterion(output, target.unsqueeze(1))  # targetに channel次元追加
        
        # Backward
        loss.backward()
        
        # Gradient clipping
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
        
        optimizer.step()
        
        total_loss += loss.item()
    
    return total_loss / len(loader)

def validate_epoch(model, loader, criterion, dice_metric, device):
    """1エポックの検証"""
    model.eval()
    total_loss = 0
    dice_scores = []
    
    with torch.no_grad():
        for batch in tqdm(loader, desc="Validation"):
            data = batch['data'].to(device)
            target = batch['target'].to(device)
            
            # Forward
            output = model(data)
            
            # Loss計算
            loss = criterion(output, target.unsqueeze(1))
            total_loss += loss.item()
            
            # Dice計算
            output_softmax = torch.softmax(output, dim=1)
            dice_metric(y_pred=output_softmax, y=target.unsqueeze(1))
    
    # Dice平均
    dice_score = dice_metric.aggregate().item()
    dice_metric.reset()
    
    return total_loss / len(loader), dice_score

In [ ]:
# 学習実行
num_epochs = 20
train_losses = []
val_losses = []
dice_scores = []
best_dice = 0

print(f"🎓 SwinUNETR v2 学習開始 ({num_epochs}エポック)\n")

for epoch in range(num_epochs):
    print(f"\nEpoch {epoch + 1}/{num_epochs}")
    print("-" * 50)
    
    # 学習
    train_loss = train_epoch(model, train_loader, criterion, optimizer, device)
    train_losses.append(train_loss)
    
    # 検証
    val_loss, dice_score = validate_epoch(model, val_loader, criterion, dice_metric, device)
    val_losses.append(val_loss)
    dice_scores.append(dice_score)
    
    # スケジューラー
    scheduler.step()
    
    print(f"Train Loss: {train_loss:.4f}")
    print(f"Val Loss: {val_loss:.4f}")
    print(f"Dice Score: {dice_score:.4f}")
    print(f"LR: {optimizer.param_groups[0]['lr']:.6f}")
    
    # ベストモデル保存
    if dice_score > best_dice:
        best_dice = dice_score
        torch.save({
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'dice_score': dice_score,
            'val_loss': val_loss,
        }, 'best_swinunetr_v2.pth')
        print("✅ ベストモデル保存")

print(f"\n🎉 学習完了!")
print(f"Best Dice Score: {best_dice:.4f}")

## 📈 Step 8: 結果可視化

In [ ]:
# 学習曲線
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

# Loss
axes[0].plot(train_losses, label='Train Loss', linewidth=2)
axes[0].plot(val_losses, label='Val Loss', linewidth=2)
axes[0].set_xlabel('Epoch')
axes[0].set_ylabel('Loss')
axes[0].set_title('Training Progress - Loss')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Dice Score
axes[1].plot(dice_scores, label='Dice Score', linewidth=2, color='green')
axes[1].set_xlabel('Epoch')
axes[1].set_ylabel('Dice Score')
axes[1].set_title('Validation Dice Score')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

# Combined
ax1 = axes[2]
color = 'tab:red'
ax1.set_xlabel('Epoch')
ax1.set_ylabel('Loss', color=color)
ax1.plot(val_losses, color=color, linewidth=2)
ax1.tick_params(axis='y', labelcolor=color)

ax2 = ax1.twinx()
color = 'tab:blue'
ax2.set_ylabel('Dice Score', color=color)
ax2.plot(dice_scores, color=color, linewidth=2)
ax2.tick_params(axis='y', labelcolor=color)
axes[2].set_title('Val Loss vs Dice Score')
axes[2].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 🔍 Step 9: 予測可視化

In [ ]:
# 予測実行
model.eval()
sample_batch = next(iter(val_loader))
data = sample_batch['data'].to(device)
target = sample_batch['target'].to(device)

with torch.no_grad():
    output = model(data)
    pred = output.argmax(dim=1)

# CPU移動
data_cpu = data.cpu().numpy()
target_cpu = target.cpu().numpy()
pred_cpu = pred.cpu().numpy()

# 3つのスライスを表示
slices = [data_cpu.shape[-1]//4, data_cpu.shape[-1]//2, 3*data_cpu.shape[-1]//4]
idx = 0

fig, axes = plt.subplots(3, 3, figsize=(15, 15))

for i, slice_idx in enumerate(slices):
    # Input
    axes[i, 0].imshow(data_cpu[idx, 0, :, :, slice_idx], cmap='gray')
    axes[i, 0].set_title(f'Input (Slice {slice_idx})')
    axes[i, 0].axis('off')
    
    # Ground Truth
    axes[i, 1].imshow(target_cpu[idx, :, :, slice_idx], cmap='tab20')
    axes[i, 1].set_title(f'Ground Truth')
    axes[i, 1].axis('off')
    
    # Prediction
    axes[i, 2].imshow(pred_cpu[idx, :, :, slice_idx], cmap='tab20')
    axes[i, 2].set_title(f'SwinUNETR v2 Prediction')
    axes[i, 2].axis('off')

plt.suptitle('SwinUNETR v2 - 3D Segmentation Results', fontsize=16)
plt.tight_layout()
plt.show()

# Dice係数
intersection = ((pred_cpu == 1) & (target_cpu == 1)).sum()
union = (pred_cpu == 1).sum() + (target_cpu == 1).sum()
dice = 2.0 * intersection / (union + 1e-8)
print(f"\n📊 サンプルDice係数: {dice:.4f}")

## 💾 Step 10: モデル保存と情報

In [ ]:
# 最終モデル保存
torch.save({
    'model_state_dict': model.state_dict(),
    'optimizer_state_dict': optimizer.state_dict(),
    'train_losses': train_losses,
    'val_losses': val_losses,
    'dice_scores': dice_scores,
    'config': {
        'model': 'SwinUNETR v2',
        'volume_size': (96, 96, 96),
        'batch_size': 1,
        'num_epochs': num_epochs,
        'best_dice': best_dice,
        'feature_size': 48,
        'data_type': 'real' if train_dataset.data_path else 'demo'
    }
}, 'final_swinunetr_v2.pth')

print("="*60)
print("🏆 SwinUNETR v2 学習完了")
print("="*60)
print("\n📊 最終結果:")
print(f"   モデル: SwinUNETR v2 (Medical AI)")
print(f"   データタイプ: {'実データ' if train_dataset.data_path else 'デモデータ'}")
print(f"   総パラメータ: {total_params:,}")
print(f"   最終Train Loss: {train_losses[-1]:.4f}")
print(f"   最終Val Loss: {val_losses[-1]:.4f}")
print(f"   最終Dice Score: {dice_scores[-1]:.4f}")
print(f"   ベストDice Score: {best_dice:.4f}")

print("\n💾 保存ファイル:")
print("   - best_swinunetr_v2.pth (ベストモデル)")
print("   - final_swinunetr_v2.pth (最終モデル)")
print("="*60)

## 🎯 完了！

### ✅ **SwinUNETR v2 (Medical AI) 学習完了**

このノートブックは：
- **完全独立** - 1ファイルで完結
- **SwinUNETR v2** - Medical AI最新アーキテクチャ
- **実データ対応** - Kaggle自動ダウンロード
- **Runpods最適化** - workspace環境対応

### 🏗️ モデル仕様:
- **アーキテクチャ**: Swin Transformer + U-Net
- **3D対応**: 96×96×96ボリューム
- **Vision Transformer**: 階層的Window Attention
- **Medical AI仕様**: DiceCELoss + AdamW

### 🚀 次のステップ:
1. より大きなfeature_size（48→72）で精度向上
2. データ拡張追加（回転、フリップ、ノイズ）
3. マルチGPU学習
4. テストデータ推論とKaggle提出